In [2]:
import pandas as pd

# Load dataset
df = pd.read_excel("/content/delivery_sample_300.xlsx")

# --- Step 1: Convert dates ---
df['Promised_Date'] = pd.to_datetime(df['Promised_Date'], errors='coerce')
df['Delivery_Date'] = pd.to_datetime(df['Delivery_Date'], errors='coerce')

# --- Step 2: Handle missing values ---
# Replace missing Delivery_Date with NaT (already handled by errors='coerce')
# Drop rows where both dates are missing
df = df.dropna(subset=['Promised_Date', 'Delivery_Date'], how='all')

# --- Step 3: Create new fields ---
df['Delay_Days'] = (df['Delivery_Date'] - df['Promised_Date']).dt.days

# On-Time Flag
df['On_Time_Flag'] = df['Delay_Days'].apply(
    lambda x: 'On Time' if pd.notnull(x) and x <= 0 else ('Late' if pd.notnull(x) else 'Unknown')
)

# Delay Bucket
def bucket_delay(x):
    if pd.isna(x):
        return "Unknown"
    elif x <= 0:
        return "On Time"
    elif x <= 2:
        return "1–2 Days Late"
    elif x <= 5:
        return "3–5 Days Late"
    else:
        return "6+ Days Late"

df['Delay_Bucket'] = df['Delay_Days'].apply(bucket_delay)

# --- Step 4: Handle outliers ---
# Cap extreme delays at 15 days for analysis
df['Delay_Capped'] = df['Delay_Days'].apply(lambda x: min(x, 15) if pd.notnull(x) else x)

# Save cleaned dataset
df.to_csv("delivery_delay_cleaned.csv", index=False)

print("✅ Data cleaned and saved as delivery_delay_cleaned.csv")

✅ Data cleaned and saved as delivery_delay_cleaned.csv
